### HyDE
🧠 What is HyDe?

HyDE (Hypothetical Document Embeddings) is a retrieval technique where, instead of embedding the user’s query directly, you first generate a hypothetical answer (document) to the query using an LLM — and then embed that hypothetical document to search your vector store.

➡️ HyDE bridges the gap between user intent and relevant content, especially when:

1. Queries are short
2. Language mismatch between query and documents
3.You want to retrieve based on answer content, not question words

In [6]:
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import init_chat_model

In [ ]:
#1. Load and chunk your dataset
chunk_size = 300
chunk_overlap = 100

#loading data
loader = WikipediaLoader(query="Steve Jobs", load_max_docs=5)
documents = loader.load()

#text splitting 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap = chunk_overlap)
docs = text_splitter.split_documents(documents = documents)
docs

[Document(metadata={'title': 'Steve Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers. \nJobs saw the commercial potential 

In [5]:
#2.Build vector store

from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorestore = FAISS.from_documents(docs, embeddings)

In [7]:
#3. Setup the LLM you;ll use to generate hypothetical answers
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = init_chat_model("openai:gpt-4o-mini")

In [9]:
from langchain.vectorstores import Chroma
##creating vectore store
db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory='output/steve_jobs')
#create base retriever
base_retriever = db.as_retriever(search_kwargs = {"k":5})

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate
#Generating a prompt for generating HyDE

def get_hyde_doc(query):
    template = """Imagine you are an expert writing a detailed explantion on the topic: '{query}
    create a hypothetical answer for the topic"""

    system_message_prompt = SystemMessagePromptTemplate.from_template(template=template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
    messages = chat_prompt.format_prompt(query=query).to_messages()
    print(messages)
    response = llm.invoke(messages)
    hypo_doc = response.content
    return hypo_doc

In [13]:
query = "When was Steve Jobs fired from Apple?"
print(get_hyde_doc(query=query))

[SystemMessage(content="Imagine you are an expert writing a detailed explantion on the topic: 'When was Steve Jobs fired from Apple?\n    create a hypothetical answer for the topic", additional_kwargs={}, response_metadata={})]
Steve Jobs was famously fired from Apple in 1985, a pivotal moment not just in his career but in the history of the technology industry. To understand the implications and context of this event, we need to look at the circumstances leading up to it.

By the early 1980s, Steve Jobs was a co-founder of Apple Computer, Inc. and had been instrumental in the launch of the Macintosh, which was released in 1984. Despite its innovative design and user-friendly interface, the Macintosh initially struggled in the marketplace, struggling against competitors like IBM. Apple’s board of directors, concerned about the company's financial performance and market position, began to lose confidence in Jobs' leadership.

As the company’s internal dynamics shifted, a power struggle 

In [14]:
matched_doc = base_retriever.invoke(get_hyde_doc(query))
print(matched_doc)

[SystemMessage(content="Imagine you are an expert writing a detailed explantion on the topic: 'When was Steve Jobs fired from Apple?\n    create a hypothetical answer for the topic", additional_kwargs={}, response_metadata={})]
[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Steve_Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to f

### Langchain-HypotheticalDocumentEmbedder

In [17]:
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder

from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

# Step 1: Load and split documents
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [19]:
# Step2: Setup LLM and embeddings 
base_embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

According to the official documentation and LangChain source code (mapping in PROMPT_MAP), the default options are:

- web_search
- sci_fact
- arguana
- trec_covid
- fiqa
- dbpedia_entity
- trec_news
- mr_tydi


In [20]:
#Step 3: HyDE Embedder using prompt_key='websearch'

hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm, 
    base_embeddings=base_embeddings,
    prompt_key='web_search'
)

In [21]:
#Step 4: Store documents in Chroma with HyDE embeddings 
vectorstore = Chroma.from_documents(
    documents=chunks, 
    embedding=hyde_embedding_function, 
    persist_directory="output/langchain"
)

In [22]:
#Step 5: RAG answer generation prompt
rag_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
rag_chain = create_stuff_documents_chain(llm=llm, prompt=rag_prompt)

In [23]:
#Step6: Final RAG pipeline
def hyde_rag_pipeline(query):
    matched_doc = vectorestore.similarity_search(query, k=4)
    print(matched_doc)
    response = rag_chain.invoke({
        "input":query,
        "context":matched_doc
    })
    return response

In [25]:
#Step 7: Run example query
query = "What memory modules does Langchain provide?"
answer = hyde_rag_pipeline(query)
print("Final answer: ", answer)

[Document(id='60c096f7-e8a9-416e-8fde-aafac0a494dc', metadata={'title': 'Steve Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomp

### Custome Prompt

In [26]:
from langchain.prompts import PromptTemplate
custom = PromptTemplate.from_template(
    "Generate a concise hypothetical answer for this topic :{query}"
)

hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm = llm, 
    base_embeddings=base_embeddings,
    custom_prompt=custom
)